In [9]:
# del session.logger
# del session

import multiprocessing

from numpy import False_

from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
from config import SpectralConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SpectralConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_group_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type="fourier",
    merger_emb_dim=2048,
    merger_channels=256,
    merger_dropout=True,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    cnn_channels=[384, 384, 384, 384, 384, 384],
    kernel_size=5,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    group_norm=True,
    half=True,
    cnn_pos_encoding=True,
    bins=16,
    hop_length=2,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

config.batch_size = 16
config.learning_rate = 3e-5
config.new_freq = 1000

config.brain_encoder_config.half = False
config.brain_encoder_config.bins = 64
config.brain_encoder_config.hop_length = 10
config.brain_encoder_config.cnn_channels = [384, 256, 256, 128, 128, 64, 32, 16, 6]

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/spectrogram/1000hz_64bins",
    clear_cache=False,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SpectralConv initialized with 9811521 parameters, cond: ['study', 'subject']
Merger True, merger channels 256
ConvBlocks: [384, 256, 256, 128, 128, 64, 32, 16, 6], hidden_dim: 384, params 8789824


2025-01-19 18:08:38,502	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1:   0%|          | 0/135 [00:00<?, ?it/s](raylet) Spilled 3788 MiB, 1 objects, write throughput 737 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 15152 MiB, 4 objects, write throughput 723 MiB/s.
(raylet) Spilled 18940 MiB, 5 objects, write throughput 683 MiB/s.


Error in processing Gwilliams2023 24 1 1.
Error in epoch 1, Gwilliams2023 24 1 1. Skipping. NaNs found in input spectrogram after merge


ValueError: NaNs found in input spectrogram after merge

(raylet) Spilled 26516 MiB, 7 objects, write throughput 571 MiB/s.
(raylet) Spilled 37880 MiB, 10 objects, write throughput 575 MiB/s.
(raylet) Spilled 77689 MiB, 28 objects, write throughput 1011 MiB/s.


: 